In [17]:
#Nur benötigt für GoLab
!pip3 install tira ir-datasets python-terrier
#Install plugin für den Stemmer
!pip install nltk

In [18]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
from nltk.stem import SnowballStemmer

import pyterrier as pt
import pandas as pd


# PyTerrier initialisieren
ensure_pyterrier_is_loaded()
tira = Client()

# PyTerrier starten
if not pt.started():
    pt.init()

# NLTK SnowballStemmer initialisieren
stemmer = SnowballStemmer("english")

# Pandas Display-Einstellungen
pd.set_option('display.max_colwidth', 0)


In [19]:
# Dataset von der TIRA Plattform abrufen
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# Vorverarbeitung: SnowballStemmer auf den Text anwenden
def stem_text(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

# Iterieren über den Corpus und Texte stemmen
processed_corpus = []
for doc in pt_dataset.get_corpus_iter():
    doc_id = doc['docno']
    text = doc['text']
    stemmed_text = stem_text(text)
    processed_corpus.append({'docno': doc_id, 'text': stemmed_text})

# Konvertieren in DataFrame
processed_corpus_df = pd.DataFrame(processed_corpus)


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [03:04<00:00, 689.37it/s] 


In [20]:
# Index erstellen
indexer = pt.IterDictIndexer("/tmp/index2", overwrite=True, stemmer='none')  # Kein zusätzlicher Stemmer hier, da wir bereits gestemmt haben
index_ref = indexer.index(processed_corpus_df.to_dict(orient='records'))

# Index laden
index = pt.IndexFactory.of(index_ref)

# BM25 Retrieval Modell initialisieren
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

11:54:48.746 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed
11:54:54.999 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents


In [21]:
#print('Now we do the retrieval...')
run = bm25(pt_dataset.get_topics('text'))

#print('Done. Here are the first 10 entries of the run')
run.head(10)

,qid,docid,docno,rank,score,query
0,1,36009,T75-2025,0,21.580689,retrieval system improving effectiveness
1,1,126778,1999.tois_journal-i,1,18.178801,retrieval system improving effectiveness
2,1,74020,2008.ntcir_workshop,2,17.750000,retrieval system improving effectiveness
3,1,1177,W18-5301,3,17.541705,retrieval system improving effectiveness
4,1,82900,2013.sigirconf_conf,4,17.028760,retrieval system improving effectiveness
5,1,92066,2005.ecir_conferenc,5,16.361147,retrieval system improving effectiveness
6,1,80745,2008.sigirconf_conf,6,15.878637,retrieval system improving effectiveness
7,1,76958,2013.dir_workshop-2,7,15.286855,retrieval system improving effectiveness
8,1,90861,2021.ecir_conferenc,8,15.064785,retrieval system improving effectiveness
9,1,29696,P07-2025,9,14.848132,retrieval system improving effectiveness


In [26]:
# Output runfile für das Deployment auf TIRA
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../../runs')


The run file is normalized outside the TIRA sandbox, I will store it at "../../runs".
Done. run file is stored under "../../runs/run.txt".
